In [ ]:
import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error

from tqdm.notebook import tqdm
import os, gc
import random
from PIL import Image
import tifffile as tiff
import cv2
import zipfile
import collections
from PIL import Image
from sklearn import preprocessing
from random import randint
from glob import glob
import shutil

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss

import torchvision
from torchvision import transforms
import albumentations as A
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import CosineAnnealingLR

import warnings
warnings.filterwarnings("ignore")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 2020
seed_everything(seed)
sz = 224
NFOLDS = 5

#ImageNet
mean = np.array([[[0.485, 0.456, 0.406]]])
std = np.array([[[0.229, 0.224, 0.225]]])

In [ ]:
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_ids = test_df.Id.to_list()
test_dir = "/kaggle/input/petfinder-pawpularity-score/test/"
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')

In [ ]:
!python detect.py\
--weights /kaggle/input/ultralyticsyolov5aweights/yolov5x.pt\
--class 15 16\
--img 512\
--conf 0.3\
--iou 0.5\
--source $test_dir\
--name inference\
--save-txt --save-conf --exist-ok

In [ ]:
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

In [ ]:
for n, image_id in tqdm(enumerate(test_ids)):
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    try:
        file_path = f'/kaggle/working/yolov5/runs/detect/inference/labels/{image_id}.txt'
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        for i, d in enumerate(data):
            xmin.append((d[2]-d[4]/2)*width)
            ymin.append((d[3]-d[5]/2)*height)
            xmax.append((d[2]+d[4]/2)*width)
            ymax.append((d[3]+d[5]/2)*height)
        all_xmin = int(min(xmin))
        all_ymin = int(min(ymin))
        all_xmax = int(max(xmax))
        all_ymax = int(max(ymax))
        all_width_half = (all_xmax - all_xmin) // 2
        all_height_half = (all_ymax - all_ymin) // 2
        r = np.maximum(all_width_half, all_height_half)
        all_xc = (all_xmin + all_xmax) // 2
        all_yc = (all_ymin + all_ymax) // 2
        final_xmin = np.maximum(all_xc-r, 0)
        final_ymin = np.maximum(all_yc-r, 0)
        final_xmax = np.minimum(all_xc+r, width)
        final_ymax = np.minimum(all_yc+r, height)
        crop_img = orig_image[final_ymin:final_ymax, final_xmin:final_xmax, :]
        crop_img = cv2.resize(crop_img, (sz, sz)).astype(np.uint8)
        np.save(save_dir + f'{image_id}', crop_img)
    except:
        orig_image = cv2.resize(orig_image, (sz, sz)).astype(np.uint8)
        np.save(save_dir + f'{image_id}', orig_image)

In [ ]:
!rm -r /kaggle/working/yolov5

In [ ]:
print(len(os.listdir("./crop_images")))
print(os.listdir("./crop_images")[:1])

In [ ]:
import sys

sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
# Asthetics
import warnings
import sklearn.exceptions

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import os
import glob
import random
import cv2

pd.set_option('display.max_columns', None)

# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Random Seed Initialize
RANDOM_SEED = 2021


def seed_everything(seed=RANDOM_SEED):
	os.environ['PYTHONHASHSEED'] = str(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = True


seed_everything()

# Device Optimization
if torch.cuda.is_available():
	device = torch.device('cuda')
else:
	device = torch.device('cpu')

print(f'Using device: {device}')

csv_dir = '../input/petfinder-pawpularity-score'
test_dir = './crop_images'
models_dir = '../input/swin-transformenrs-pet-net'

test_file_path = os.path.join(csv_dir, 'test.csv')
sample_sub_file_path = os.path.join(csv_dir, 'sample_submission.csv')
print(f'Test file: {test_file_path}')
print(f'Models path: {models_dir}')

test_df = pd.read_csv(test_file_path)
sample_df = pd.read_csv(sample_sub_file_path)


def return_filpath(name, folder):
	path = os.path.join(folder, f'{name}.npy')
	return path


test_df['image_path'] = test_df['Id'].apply(lambda x: return_filpath(x, folder=test_dir))
test_df.head()

df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
print('Train shape:', df.shape)
df.head()

params = {
	'model': 'swin_large_patch4_window12_384',
	'model2': 'swin_large_patch4_window7_224',
	'dense_features': ['Subject Focus', 'Eyes', 'Face', 'Near',
	                   'Action', 'Accessory', 'Group', 'Collage',
	                   'Human', 'Occlusion', 'Info', 'Blur'],
	'pretrained': False,
	'inp_channels': 3,
	'im_size': 384,
	'device': device,
	'batch_size': 8,
	'num_workers': 0,
	'out_features': 1,
	'debug': False
}
if params['debug']:
	test_df = test_df.sample(frac=0.1)


def get_test_transforms(DIM=params['im_size']):
	return albumentations.Compose(
		[
			albumentations.Resize(DIM, DIM),
			albumentations.Normalize(
				mean=[0.485, 0.456, 0.406],
				std=[0.229, 0.224, 0.225],
			),
			ToTensorV2(p=1.0)
		]
	)


def get_test_Flip_transforms(DIM=params['im_size']):
	return albumentations.Compose(
		[albumentations.HorizontalFlip(p=0.5),

         albumentations.VerticalFlip(p=0.5),
		 albumentations.Resize(DIM, DIM),
		 albumentations.Normalize(
			 mean=[0.485, 0.456, 0.406],
			 std=[0.229, 0.224, 0.225],
		 ),
		 ToTensorV2(p=1.0)
		 ]
	)


def get_test_Shift_Scale_transforms(DIM=params['im_size']):
	return albumentations.Compose(
		[albumentations.ShiftScaleRotate(
			shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5
		),

			albumentations.Resize(DIM, DIM),
			albumentations.Normalize(
				mean=[0.485, 0.456, 0.406],
				std=[0.229, 0.224, 0.225],
			),
			ToTensorV2(p=1.0)
		]
	)


def get_test_Rotate_transforms(DIM=params['im_size']):
	return albumentations.Compose(
		[albumentations.Rotate(limit=180, p=0.7),
		 albumentations.Resize(DIM, DIM),
		 albumentations.Normalize(
			 mean=[0.485, 0.456, 0.406],
			 std=[0.229, 0.224, 0.225],
		 ),
		 ToTensorV2(p=1.0)
		 ]
	)


class CuteDataset(Dataset):
	def __init__(self, images_filepaths, dense_features, targets, transform=None, transform_flip=None,
	             transform_shiftscale=None, transform_rotate=None):
		self.images_filepaths = images_filepaths
		self.dense_features = dense_features
		self.targets = targets
		self.transform = transform
		self.transform_flip = transform_flip
		self.transform_shiftscale = transform_shiftscale
		self.transform_rotate = transform_rotate

	def __len__(self):
		return len(self.images_filepaths)

	def __getitem__(self, idx):
		image_filepath = self.images_filepaths[idx]
		image = np.load(image_filepath).astype(np.float32)

		if self.transform is not None:
			image1 = self.transform(image=image)['image']
		if self.transform_flip is not None:
			image2 = self.transform_flip(image=image)['image']
		if self.transform_shiftscale is not None:
			image3 = self.transform_shiftscale(image=image)['image']
		if self.transform_shiftscale is not None:
			image4 = self.transform_rotate(image=image)['image']

		dense = self.dense_features[idx, :]
		label = torch.tensor(self.targets[idx]).float()
		return image1, image2, image3, image4, dense, label


class PetNet(nn.Module):
	def __init__(self, model_name=params['model'], out_features=params['out_features'],
	             inp_channels=params['inp_channels'],
	             pretrained=params['pretrained'], num_dense=len(params['dense_features'])):
		super().__init__()
		self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels)
		n_features = self.model.head.in_features
		self.model.head = nn.Linear(n_features, 128)
		self.fc = nn.Sequential(
			nn.Linear(128 + num_dense, 64),
			nn.ReLU(),
			nn.Linear(64, out_features)
		)
		self.dropout = nn.Dropout(0.2)

	def forward(self, image, dense):
		embeddings = self.model(image)
		x = self.dropout(embeddings)
		x = torch.cat([x, dense], dim=1)
		output = self.fc(x)
		return output


class PetNet2(nn.Module):
	def __init__(self, model_name=params['model2'], out_features=params['out_features'],
	             inp_channels=params['inp_channels'],
	             pretrained=params['pretrained'], num_dense=len(params['dense_features'])):
		super().__init__()
		self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels)
		n_features = self.model.head.in_features
		self.model.head = nn.Linear(n_features, 128)
		self.fc = nn.Sequential(
			nn.Linear(128, 64),
			nn.ReLU(),
			nn.Linear(64, out_features)
		)
		self.dropout = nn.Dropout(0.2)

	def forward(self, image, dense):
		embeddings = self.model(image)
		x = self.dropout(embeddings)
		x = torch.cat([x], dim=1)
		output = self.fc(x)
		return output




predicted_labels = None
for model_name in glob.glob("../input/imagenet-1k-swin-large-384/SwinLarge384Withcrop" + '/*.pth'):
	model = PetNet()
	model.load_state_dict(torch.load(model_name))
	model = model.to(params['device'])
	model.eval()

	test_dataset = CuteDataset(
		images_filepaths=test_df['image_path'].values,
		dense_features=test_df[params['dense_features']].values,
		targets=sample_df['Pawpularity'].values,
		transform=get_test_transforms(),
		transform_flip=get_test_Flip_transforms(),
		transform_shiftscale=get_test_Shift_Scale_transforms(),
		transform_rotate=get_test_Rotate_transforms()

	)
	test_loader = DataLoader(
		test_dataset, batch_size=params['batch_size'],
		shuffle=False, num_workers=params['num_workers'],
		pin_memory=True
	)

	temp_preds = None
	with torch.no_grad():
		for (images, images_flip, images_shift, images_rotate, dense, target) in tqdm(test_loader,
		                                                                              desc=f'Predicting. '):
			images = images.to(params['device'], non_blocking=True)
			images_flip = images_flip.to(params['device'], non_blocking=True)
			images_shift = images_shift.to(params['device'], non_blocking=True)
			images_rotate = images_rotate.to(params['device'], non_blocking=True)
			dense = dense.to(params['device'], non_blocking=True)
			predictions = torch.sigmoid(model(images, dense)).to('cpu').numpy() * 100
			predictions += torch.sigmoid(model(images_flip, dense)).to('cpu').numpy() * 100
			predictions += torch.sigmoid(model(images_shift, dense)).to('cpu').numpy() * 100
			predictions += torch.sigmoid(model(images_rotate, dense)).to('cpu').numpy() * 100

			predictions = predictions / 4;print(predictions)
			if temp_preds is None:
				temp_preds = predictions
			else:
				temp_preds = np.vstack((temp_preds, predictions))

	if predicted_labels is None:
		predicted_labels = temp_preds
	else:
		predicted_labels += temp_preds

predicted_labels /= (len(glob.glob("../input/imagenet-1k-swin-large-384/SwinLarge384Withcrop" + '/*.pth')))

test_dataset = CuteDataset(
	images_filepaths=test_df['image_path'].values,
	dense_features=test_df[params['dense_features']].values,
	targets=sample_df['Pawpularity'].values,
	transform=get_test_transforms(224),
	transform_flip=get_test_Flip_transforms(224),
	transform_shiftscale=get_test_Shift_Scale_transforms(224),
	transform_rotate=get_test_Rotate_transforms(224)
)

test_loader = DataLoader(
	test_dataset, batch_size=params['batch_size'],
	shuffle=False, num_workers=params['num_workers'],
	pin_memory=True
)

predicted_labels2 = None
for model_name in glob.glob("../input/imagenet-1k-swin/SwinLargw224Nonmixp" + '/*.pth'):
	model = PetNet2()
	model.load_state_dict(torch.load(model_name))
	model = model.to(params['device'])
	model.eval()

	temp_preds = None
	with torch.no_grad():
		for (images, images_flip, images_shift, images_rotate, dense, target) in tqdm(test_loader,
		                                                                              desc=f'Predicting. '):
			images = images.to(params['device'], non_blocking=True)
			images_flip = images_flip.to(params['device'], non_blocking=True)
			images_shift = images_shift.to(params['device'], non_blocking=True)
			images_rotate = images_rotate.to(params['device'], non_blocking=True)
			dense = dense.to(params['device'], non_blocking=True)
			predictions = torch.sigmoid(model(images, dense)).to('cpu').numpy() * 100
			predictions += torch.sigmoid(model(images_flip, dense)).to('cpu').numpy() * 100
			predictions += torch.sigmoid(model(images_shift, dense)).to('cpu').numpy() * 100
			predictions += torch.sigmoid(model(images_rotate, dense)).to('cpu').numpy() * 100

			predictions = predictions / 4;print(predictions)
			if temp_preds is None:
				temp_preds = predictions
			else:
				temp_preds = np.vstack((temp_preds, predictions))

	if predicted_labels2 is None:
		predicted_labels2 = temp_preds
	else:
		predicted_labels2 += temp_preds

#     del model

predicted_labels2 /= (len(glob.glob("../input/imagenet-1k-swin/SwinLargw224Nonmixp" + '/*.pth')))


In [ ]:

import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import os
import glob
import random
import cv2
pd.set_option('display.max_columns', None)

# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Random Seed Initialize
RANDOM_SEED = 2021

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# Device Optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')


csv_dir = '../input/petfinder-pawpularity-score'
test_dir = '../input/petfinder-pawpularity-score/test'
models_dir = '../input/swin-transformenrs-pet-net'

test_file_path = os.path.join(csv_dir, 'test.csv')
sample_sub_file_path = os.path.join(csv_dir, 'sample_submission.csv')
print(f'Test file: {test_file_path}')
print(f'Models path: {models_dir}')

test_df = pd.read_csv(test_file_path)
sample_df = pd.read_csv(sample_sub_file_path)

def return_filpath(name, folder):
    path = os.path.join(folder, f'{name}.jpg')
    return path
test_df['image_path'] = test_df['Id'].apply(lambda x: return_filpath(x, folder=test_dir))
test_df.head()

df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
print('Train shape:', df.shape )
df.head()

params = {
    'model': 'swin_large_patch4_window12_384_in22k',
    'model2' : 'vit_large_patch16_224_in21k',
    'dense_features': ['Subject Focus', 'Eyes', 'Face', 'Near',
                       'Action', 'Accessory', 'Group', 'Collage',
                       'Human', 'Occlusion', 'Info', 'Blur'],
    'pretrained': False,
    'inp_channels': 3,
    'im_size': 384,
    'device': device,
    'batch_size': 16,
    'num_workers' : 0,
    'out_features': 1,
    'debug': False
}

if params['debug']:
    test_df = test_df.sample(frac=0.1)

    
def get_test_transforms(DIM = params['im_size']):
    return albumentations.Compose(
        [albumentations.HorizontalFlip(p=0.5),

         albumentations.VerticalFlip(p=0.5),
          albumentations.Resize(DIM,DIM),
          albumentations.Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225],
          ),
          ToTensorV2(p=1.0)
        ]
    )


class CuteDataset(Dataset):
    def __init__(self, images_filepaths, dense_features, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.dense_features = dense_features
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        dense = self.dense_features[idx, :]
        label = torch.tensor(self.targets[idx]).float()
        return image, dense, label

class PetNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'], inp_channels=params['inp_channels'],
                 pretrained=params['pretrained'], num_dense=len(params['dense_features'])):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, 128)
        self.fc = nn.Sequential(
            nn.Linear(128 + num_dense, 64),
            nn.ReLU(),
            nn.Linear(64, out_features)
        )
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, image, dense):
        embeddings = self.model(image)
        x = self.dropout(embeddings)
        x = torch.cat([x, dense], dim=1)
        output = self.fc(x)
        return output

test_dataset = CuteDataset(
    images_filepaths = test_df['image_path'].values,
    dense_features = test_df[params['dense_features']].values,
    targets = sample_df['Pawpularity'].values,
    transform = get_test_transforms()
)
test_loader = DataLoader(
    test_dataset, batch_size=params['batch_size'],
    shuffle=False, num_workers=params['num_workers'],
    pin_memory=True
)

predicted_labels1 = None
for model_name in glob.glob("../input/swin-transformenrs-pet-net" + '/*.pth'):
    model = PetNet()
    model.load_state_dict(torch.load(model_name))
    model = model.to(params['device'])
    model.eval()

    test_dataset = CuteDataset(
        images_filepaths = test_df['image_path'].values,
        dense_features = test_df[params['dense_features']].values,
        targets = sample_df['Pawpularity'].values,
        transform = get_test_transforms()
    )
    test_loader = DataLoader(
        test_dataset, batch_size=params['batch_size'],
        shuffle=False, num_workers=params['num_workers'],
        pin_memory=True
    )

    temp_preds = None
    with torch.no_grad():
        for (images, dense, target) in tqdm(test_loader, desc=f'Predicting. '):
            images = images.to(params['device'], non_blocking=True)
            dense = dense.to(params['device'], non_blocking=True)
            predictions = torch.sigmoid(model(images, dense)).to('cpu').numpy()*100
            
            if temp_preds is None:
                temp_preds = predictions
            else:
                temp_preds = np.vstack((temp_preds, predictions))

    if predicted_labels1 is None:
        predicted_labels1 = temp_preds
    else:
        predicted_labels1 += temp_preds
        
predicted_labels1 /= (len(glob.glob("../input/swin-transformenrs-pet-net" + '/*.pth')))

In [ ]:
predicted_labels = predicted_labels * 1/3 + predicted_labels2 * 1/3 + predicted_labels1 * 1/3

In [ ]:
print(predicted_labels2)

In [ ]:
!rm -r /kaggle/working/crop_images

In [ ]:
sub_df = pd.DataFrame()
sub_df['Id'] = test_df['Id']
sub_df['Pawpularity'] = predicted_labels
sub_df.to_csv('submission.csv', index=False)
